In [23]:
import requests
import random
import string
import os
import signal
from functools import wraps
import mimetypes
from bs4 import BeautifulSoup
import base64
import re

def generate_random_boundary():
    return ''.join(random.choices(string.digits, k=30))

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wrapper
    return decorator

def save_base64_image(code_folder, filename, base64_data):
    img_data = base64.b64decode(base64_data)
    with open(os.path.join(code_folder, filename), 'wb') as f:
        f.write(img_data)

def save_html_with_images(code, content):
    soup = BeautifulSoup(content, 'html.parser')
    img_tags = soup.find_all('img')
    
    code_folder = os.path.join('codes', code)
    os.makedirs(code_folder, exist_ok=True)
    
    for img in img_tags:
        if 'src' in img.attrs:
            src = img['src']
            if src.startswith('data:image'):
                # Handle base64 encoded image
                image_type, base64_data = src.split(',', 1)
                extension = image_type.split('/')[-1].split(';')[0]
                filename = f"image_{img_tags.index(img)}.{extension}"
                save_base64_image(code_folder, filename, base64_data)
                img['src'] = filename
            else:
                # Handle URL image
                img_url = src
                img_filename = os.path.basename(img_url)
                img_path = os.path.join(code_folder, img_filename)
                
                # Download and save the image
                img_response = requests.get(img_url)
                if img_response.status_code == 200:
                    with open(img_path, 'wb') as img_file:
                        img_file.write(img_response.content)
                    
                    # Update the img src to point to the local file
                    img['src'] = img_filename

    # Save the modified HTML
    html_filename = os.path.join(code_folder, f"{code}.html")
    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(str(soup))
    
    return html_filename

@timeout(30)  # Set timeout to 10 seconds
def check_code(code):
    url = "https://codes.thisisnotawebsitedotcom.com/"
    
    boundary_number = generate_random_boundary()
    boundary = f"-----------------------------{boundary_number}"
    
    payload = f'{boundary}\r\nContent-Disposition: form-data; name="code"\r\n\r\n{code}\r\n{boundary}--'
    
    headers = {
        'Host': 'codes.thisisnotawebsitedotcom.com',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://thisisnotawebsitedotcom.com/',
        'Origin': 'https://thisisnotawebsitedotcom.com',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:129.0) Gecko/20100101 Firefox/129.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': f'multipart/form-data; boundary={boundary[2:]}',
        'Priority': 'u=0',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache'
    }
    
    try:
        response = requests.post(url, data=payload, headers=headers)
        
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type', '')
            
            code_folder = os.path.join('codes', code)
            os.makedirs(code_folder, exist_ok=True)
            
            if content_type.startswith('image/'):
                # It's an image
                extension = mimetypes.guess_extension(content_type) or ''
                filename = os.path.join(code_folder, f"{code}{extension}")
                with open(filename, 'wb') as f:
                    f.write(response.content)
                return f"Code '{code}' exists. Image saved as {filename}"
            
            elif content_type.startswith('text/html'):
                # It's HTML content
                html_filename = save_html_with_images(code, response.text)
                return f"Code '{code}' exists. HTML content saved as {html_filename}"
            
            else:
                # It's plain text or other content
                text_filename = os.path.join(code_folder, f"{code}.txt")
                with open(text_filename, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                return f"Code '{code}' exists. Response saved as {text_filename}"
        else:
            print(response.text)
            return f"Code '{code}' does not exist. Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"An error occurred: {str(e)}"

if __name__ == "__main__":
    try:
        code_to_check = "fsadf"
        result = check_code(code_to_check)
        print(result)
    except TimeoutException:
        print(f"Request for code '{code_to_check}' timed out after 10 seconds.")

<!DOCTYPE html><head><meta name="robots" content="noindex"></head><body style="background-color: black"></body>
Code 'fsadf' does not exist. Status code: 404


In [24]:
gravity_falls_words = [
    "Dipper", "Mabel", "Stan", "Ford", "Soos", "Wendy", "Bill", "Mystery", "Waddles",
    "Weirdmageddon", "Journal", "Pacifica", "Gideon", "Gnomes", "Summerween", "Bottomless",
    "TimeBaby", "Shmebulock", "Pines", "Cipher", "Multibear", "Manotaurs", "Pterodactyl",
    "Gobblewonker", "Robbie", "Susan", "Toby", "McGucket", "Blendin", "Rumble", "Candy",
    "Grenda", "Axolotl", "Tad", "Shacktron", "Dungeons", "Smile", "Northwest", "Mermando",
    "Chutzpar", "Quentin", "Woodpecker", "Trembley", "Dip", "Grunkle", "Corduroy", "Ramirez",
    "Gleeful", "Valentino", "Determined", "Blandin", "Strange", "Dip"
]

for code in gravity_falls_words:
    try:
        result = check_code(code.lower())
        print(result)
    except TimeoutException:
        print(f"Request for code '{code}' timed out after 10 seconds.")

Code 'Dipper' exists. HTML content saved as codes/Dipper/Dipper.html
Code 'Mabel' exists. HTML content saved as codes/Mabel/Mabel.html
Code 'Stan' exists. HTML content saved as codes/Stan/Stan.html
Code 'Ford' exists. Image saved as codes/Ford/Ford.png
Code 'Soos' exists. HTML content saved as codes/Soos/Soos.html
Code 'Wendy' exists. Image saved as codes/Wendy/Wendy.png
Code 'Bill' exists. HTML content saved as codes/Bill/Bill.html
Code 'Mystery' exists. HTML content saved as codes/Mystery/Mystery.html
Code 'Waddles' exists. HTML content saved as codes/Waddles/Waddles.html
Code 'Weirdmageddon' exists. Image saved as codes/Weirdmageddon/Weirdmageddon.png
<!DOCTYPE html><head><meta name="robots" content="noindex"></head><body style="background-color: black"></body>
Code 'Journal' does not exist. Status code: 404
Code 'Pacifica' exists. Image saved as codes/Pacifica/Pacifica.png
Code 'Gideon' exists. HTML content saved as codes/Gideon/Gideon.html
<!DOCTYPE html><head><meta name="robots" 

In [28]:
import requests
import random
import string
import os
import signal
from functools import wraps
import mimetypes
from bs4 import BeautifulSoup
import base64
import re
from multiprocessing import Pool, cpu_count

def generate_random_boundary():
    return ''.join(random.choices(string.digits, k=30))

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wrapper
    return decorator

def save_base64_image(code_folder, filename, base64_data):
    img_data = base64.b64decode(base64_data)
    with open(os.path.join(code_folder, filename), 'wb') as f:
        f.write(img_data)

def save_html_with_images(code, content):
    soup = BeautifulSoup(content, 'html.parser')
    img_tags = soup.find_all('img')
    
    code_folder = os.path.join('codes', code)
    os.makedirs(code_folder, exist_ok=True)
    
    for img in img_tags:
        if 'src' in img.attrs:
            src = img['src']
            if src.startswith('data:image'):
                # Handle base64 encoded image
                image_type, base64_data = src.split(',', 1)
                extension = image_type.split('/')[-1].split(';')[0]
                filename = f"image_{img_tags.index(img)}.{extension}"
                save_base64_image(code_folder, filename, base64_data)
                img['src'] = filename
            else:
                # Handle URL image
                img_url = src
                img_filename = os.path.basename(img_url)
                img_path = os.path.join(code_folder, img_filename)
                
                # Download and save the image
                img_response = requests.get(img_url)
                if img_response.status_code == 200:
                    with open(img_path, 'wb') as img_file:
                        img_file.write(img_response.content)
                    
                    # Update the img src to point to the local file
                    img['src'] = img_filename

    # Save the modified HTML
    html_filename = os.path.join(code_folder, f"{code}.html")
    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(str(soup))
    
    return html_filename

def check_code(code):
    url = "https://codes.thisisnotawebsitedotcom.com/"
    
    boundary_number = generate_random_boundary()
    boundary = f"-----------------------------{boundary_number}"
    
    payload = f'{boundary}\r\nContent-Disposition: form-data; name="code"\r\n\r\n{code}\r\n{boundary}--'
    
    headers = {
        'Host': 'codes.thisisnotawebsitedotcom.com',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://thisisnotawebsitedotcom.com/',
        'Origin': 'https://thisisnotawebsitedotcom.com',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:129.0) Gecko/20100101 Firefox/129.0',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Content-Type': f'multipart/form-data; boundary={boundary[2:]}',
        'Priority': 'u=0',
        'Pragma': 'no-cache',
        'Cache-Control': 'no-cache'
    }
    
    try:
        response = requests.post(url, data=payload, headers=headers, timeout=30)
        
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type', '')
            
            code_folder = os.path.join('codes', code)
            os.makedirs(code_folder, exist_ok=True)
            
            if content_type.startswith('image/'):
                # It's an image
                extension = mimetypes.guess_extension(content_type) or ''
                filename = os.path.join(code_folder, f"{code}{extension}")
                with open(filename, 'wb') as f:
                    f.write(response.content)
                return f"Code '{code}' exists. Image saved as {filename}"
            
            elif content_type.startswith('text/html'):
                # It's HTML content
                html_filename = save_html_with_images(code, response.text)
                return f"Code '{code}' exists. HTML content saved as {html_filename}"
            
            else:
                # It's plain text or other content
                text_filename = os.path.join(code_folder, f"{code}.txt")
                with open(text_filename, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                return f"Code '{code}' exists. Response saved as {text_filename}"
        else:
            return f"Code '{code}' does not exist. Status code: {response.status_code}"
    except requests.RequestException as e:
        return f"An error occurred for code '{code}': {str(e)}"

def process_codes(codes):
    with Pool(processes=cpu_count()) as pool:
        results = pool.map(check_code, codes)
    return results

if __name__ == "__main__":
    # Example list of codes to check
    codes_to_check = ["fsadf", "dipper", "mabel", "stan", "soos", "wendy", "bill", "waddles"]
    
    results = process_codes(codes_to_check)
    
    for code, result in zip(codes_to_check, results):
        print(f"Result for code '{code}':")
        print(result)
        print("---")

Process SpawnPoolWorker-50:
Process SpawnPoolWorker-51:
Process SpawnPoolWorker-49:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'check_code' on <module '__main__' (built-in)>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap

KeyboardInterrupt: 